In [ ]:
from getpass import getpass
import os

api_key = getpass("Enter your GEMINI_API_KEY: ")
os.environ["GEMINI_API_KEY"] = api_key

Enter your GEMINI_API_KEY: ··········


In [ ]:
!pip install -q google-genai mcp fastmcp httpx nest_asyncio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 367.5/367.5 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.1/223.1 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.4/96.4 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.1/331.1 kB 21.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires google-auth==2.38.0, but you have google-auth 2.43.0 which is incompatible.
google-auth-oauthlib 1.2.3 requires google-auth<2.42.0,>=2.15.0, but you have google-auth 2.43.0 which is incompatible.


In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
from fastmcp import FastMCP
import httpx

mcp_server = FastMCP("weather")

@mcp_server.tool()
def get_weather(latitude: float, longitude: float) -> str:
    """3‑day min/max temperature forecast via Open‑Meteo."""
    url = (
        f"https://api.open-meteo.com/v1/forecast"
        f"?latitude={latitude}&longitude={longitude}"
        "&daily=temperature_2m_min,temperature_2m_max&timezone=UTC"
    )
    resp = httpx.get(url, timeout=10)
    daily = resp.json()["daily"]
    return "\n".join(
        f"{date}: low {mn}°C, high {mx}°C"
        for date, mn, mx in zip(
            daily["time"],
            daily["temperature_2m_min"],
            daily["temperature_2m_max"],
        )
    )

@mcp_server.tool()
def get_alerts(state: str) -> str:
    """Dummy US‑state alerts."""
    return f"No active weather alerts for {state.upper()}."

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [ ]:
import asyncio
from google import genai
from google.genai import types
from fastmcp import Client as MCPClient
from fastmcp.client.transports import FastMCPTransport

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [ ]:
client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))
MODEL = "gemini-2.0-flash"
transport = FastMCPTransport(mcp_server)

In [ ]:
function_declarations = [
    {
        "name": "get_weather",
        "description": "Return a 3‑day min/max temperature forecast for given coordinates.",
        "parameters": {
            "type": "object",
            "properties": {
                "latitude": {
                    "type": "number",
                    "description": "Latitude of target location."
                },
                "longitude": {
                    "type": "number",
                    "description": "Longitude of target location."
                }
            },
            "required": ["latitude", "longitude"]
        }
    },
    {
        "name": "get_alerts",
        "description": "Return any active weather alerts for a given U.S. state.",
        "parameters": {
            "type": "object",
            "properties": {
                "state": {
                    "type": "string",
                    "description": "Two‑letter U.S. state code, e.g. 'CA'."
                }
            },
            "required": ["state"]
        }
    }
]

tool_defs = types.Tool(function_declarations=function_declarations)

In [ ]:
async def run_gemini(lat: float, lon: float):
    async with MCPClient(transport) as mcp_client:
        prompt = f"Give me a 3‑day weather forecast for latitude={lat}, longitude={lon}."
        response = client.models.generate_content(
            model=MODEL,
            contents=[prompt],
            config=types.GenerateContentConfig(
                temperature=0,
                tools=[tool_defs]
            )
        )

        call = response.candidates[0].content.parts[0].function_call
        if not call:
            print("No function call; GPT said:", response.text)
            return

        print("🔧 Gemini wants:", call.name, call.args)

        result = await mcp_client.call_tool(call.name, call.args)
        print("\n📋 Tool result:\n", result)

asyncio.get_event_loop().run_until_complete(run_gemini(37.7749, -122.4194))

🔧 Gemini wants: get_weather {'longitude': -122.4194, 'latitude': 37.7749}

📋 Tool result:
 CallToolResult(content=[TextContent(type='text', text='2025-11-11: low 12.5°C, high 25.5°C\n2025-11-12: low 12.3°C, high 17.9°C\n2025-11-13: low 13.3°C, high 16.9°C\n2025-11-14: low 13.2°C, high 17.0°C\n2025-11-15: low 12.6°C, high 16.8°C\n2025-11-16: low 13.8°C, high 14.4°C\n2025-11-17: low 12.2°C, high 16.6°C', annotations=None, meta=None)], structured_content={'result': '2025-11-11: low 12.5°C, high 25.5°C\n2025-11-12: low 12.3°C, high 17.9°C\n2025-11-13: low 13.3°C, high 16.9°C\n2025-11-14: low 13.2°C, high 17.0°C\n2025-11-15: low 12.6°C, high 16.8°C\n2025-11-16: low 13.8°C, high 14.4°C\n2025-11-17: low 12.2°C, high 16.6°C'}, data='2025-11-11: low 12.5°C, high 25.5°C\n2025-11-12: low 12.3°C, high 17.9°C\n2025-11-13: low 13.3°C, high 16.9°C\n2025-11-14: low 13.2°C, high 17.0°C\n2025-11-15: low 12.6°C, high 16.8°C\n2025-11-16: low 13.8°C, high 14.4°C\n2025-11-17: low 12.2°C, high 16.6°C', is_er

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
